In [1]:
from dataclasses import dataclass
import pandas as pd
from pydantic import BaseModel
from typing import Dict, Any, Optional, List
from mlserver.codecs import PandasCodec
from abc import ABC, abstractmethod
from pathlib import Path
import logging

log = logging.getLogger("model-settings")

class Settings(BaseModel):
    """Settings for the MLServer Settings Service."""
    debug:bool = True
    
class ColumnData(BaseModel):
    """Column Data is a json representation of dataframe column"""
    name:str
    datatype:str = None
    parameters:Optional[Dict[str, Any]] = None
    shape:Optional[List[int]] = None
    data:List = None
    
class ColumnMeta(BaseModel):
    """Column Meta is a description of dataframe column"""
    name:str
    datatype:str = None
    parameters:Optional[Dict[str, Any]] = None
    shape:Optional[List[int]] = [-1]

class ModelParams(BaseModel):
    """Used in model-settings"""
    uri:str
    version:str = "v0.1.0"
    
class ModelSettings(BaseModel):
    """A model settings includes a model name, implementation method, 
        parameters(model file location etc.) and input formats."""
    name:str
    implementation:str
    parameters:ModelParams
    inputs:List[ColumnMeta] = None

class InputRequest(BaseModel):
    """Tell me the content type of the entire input, and give me the real request data"""
    content_type:Optional[str] = None
    parameters:Optional[dict] = None
    inputs:List[ColumnData]


class SettingsCreator(ABC):
    """Abstract class for creating model settings object"""

    @abstractmethod
    def create_model_settings(self) -> ModelSettings:
        pass

    def create_settings(self) -> Settings:
        return Settings(debug = True)

    def dump_json(self, path = "."):
        settings_path = Path(path) / "settings.json"
        model_settings_path = Path(path) / "model-settings.json"
        
        with open(settings_path, 'w') as f:
            f.write(self.create_settings().json(indent = 2))
            log.info(f"Write settings to {settings_path}")

        with open(model_settings_path, 'w') as f:
            f.write(self.create_model_settings().json(indent = 2))
            log.info(f"Write model settings to {model_settings_path}")


@dataclass
class SklearnModelSettings(SettingsCreator):

    name:str
    uri:str
    df:pd.DataFrame
    version:str = "v0.1.0"
    implementation:str = 'mlserver_sklearn.SKLearnModel'

    def create_inputs_meta(self):
        """Create metadata input for dataframe"""
        request = PandasCodec.encode_request(self.df.head(1))
        inputs = []
        for r in request.inputs:
            col_meta = {}
            if r.datatype == 'BYTES':
                col_meta['parameters'] = {'content_type':'str'}
            col_meta['name'] = r.name
            col_meta['datatype'] = r.datatype
            inputs.append(col_meta)
        return inputs

    def create_model_settings(self) -> ModelSettings:
        inputs = self.create_inputs_meta()
        model_settings = ModelSettings(
            name = self.name,
            implementation = self.implementation,
            parameters = ModelParams(uri = self.uri, version = self.version),
            inputs = inputs
        )
        return model_settings


In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn import svm
import pandas as pd
import joblib

iris = load_iris()
y = iris['target']
X = iris['data']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
feature_names = iris.feature_names
df = pd.DataFrame(X, columns = feature_names)

In [25]:

def create_request(df,n = 3):
    req = PandasCodec.encode_request(df.head(n))
    req.parameters = {'content_type':"pd"}
    return req.json(include = {'parameters','inputs'})
    
create_request(df)

'{"parameters": {"content_type": "pd"}, "inputs": [{"name": "sepal length (cm)", "shape": [3], "datatype": "FP64", "parameters": null, "data": [5.1, 4.9, 4.7]}, {"name": "sepal width (cm)", "shape": [3], "datatype": "FP64", "parameters": null, "data": [3.5, 3.0, 3.2]}, {"name": "petal length (cm)", "shape": [3], "datatype": "FP64", "parameters": null, "data": [1.4, 1.4, 1.3]}, {"name": "petal width (cm)", "shape": [3], "datatype": "FP64", "parameters": null, "data": [0.2, 0.2, 0.2]}]}'